In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import pandas as pd
import numpy as np
import os 
import re
import operator
import pickle
import nltk 
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer



In [ ]:
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/IR/article.csv"
df = pd.read_csv(path)


In [ ]:
df.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [ ]:
df_final = df[(df['content'].str.split().str.len() > 250) & (df['content'].str.split().str.len() < 300)][['title', 'content']]
df_final

,title,content
19,Fecal Pollution Taints Water at Melbourne’s Be...,"SYDNEY, Australia — The annual beach pilgri..."
203,Smithsonian Hears Call for Women’s Museum - Th...,"Leading women from politics, the arts and othe..."
439,NBC Will Bring Back ‘Will &amp Grace’ - The Ne...,"In an era of revivals and reboots, this may ha..."
571,Bernie Ecclestone Is Replaced as Formula One C...,LONDON — Bernie Ecclestone’s longstanding l...
616,Pence Will Speak at Anti-Abortion Rally - The ...,WASHINGTON — Vice President Mike Pence will...
...,...,...
141971,I interviewed the Ohio State attacker on the f...,Abdul Razak Ali Artan was sitting alone a...
142082,The Nationals have two years to win a World Se...,With baseball’s winter meetings right here al...
142279,Traces of explosives found on human remains fr...,CAIRO — Traces of explosives have been fo...
142407,Australian police say they prevented bombings ...,"MELBOURNE, Australia — Police arrested sev..."


In [ ]:
df_final.reset_index(drop=True, inplace = True)
df_final

,title,content
0,Fecal Pollution Taints Water at Melbourne’s Be...,"SYDNEY, Australia — The annual beach pilgri..."
1,Smithsonian Hears Call for Women’s Museum - Th...,"Leading women from politics, the arts and othe..."
2,NBC Will Bring Back ‘Will &amp Grace’ - The Ne...,"In an era of revivals and reboots, this may ha..."
3,Bernie Ecclestone Is Replaced as Formula One C...,LONDON — Bernie Ecclestone’s longstanding l...
4,Pence Will Speak at Anti-Abortion Rally - The ...,WASHINGTON — Vice President Mike Pence will...
...,...,...
7390,I interviewed the Ohio State attacker on the f...,Abdul Razak Ali Artan was sitting alone a...
7391,The Nationals have two years to win a World Se...,With baseball’s winter meetings right here al...
7392,Traces of explosives found on human remains fr...,CAIRO — Traces of explosives have been fo...
7393,Australian police say they prevented bombings ...,"MELBOURNE, Australia — Police arrested sev..."


In [ ]:
df_final['content'] = df_final['content'].str.replace(r'[^\w\s]+', '')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
# ! curl -L -o 4.tar.gz "https://tfhub.dev/google/universal-sentence-encoder/4?tf-hub-format=compressed"


In [ ]:
# #Model load through local path:
# module_path ="https://tfhub.dev/google/universal-sentence-encoder/4"
# %time model = hub.load(module_path)
# #Create function for using model training
# def embed(input):
#     return model(input)

In [ ]:
# Model_USE= embed(df.content[0:2500])


In [ ]:
# exported = tf.train.Checkpoint(v=tf.Variable(Model_USE))
# exported.f = tf.function(
#     lambda  x: exported.v * x,
#     input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])
# tf.saved_model.save(exported,'/content/drive/MyDrive/IR/MODEL')

In [ ]:
# imported = tf.saved_model.load('/content/drive/MyDrive/IR/MODEL/')
# loadedmodel =imported.v.numpy()

In [ ]:
# def SearchDocument(query):
#     q =[query]
#     # embed the query for calcluating the similarity
#     Q_Train =embed(q)
    
#     #imported_m = tf.saved_model.load('/home/zettadevs/GoogleUSEModel/TrainModel')
#     #loadedmodel =imported_m.v.numpy()
#     # Calculate the Similarity
#     linear_similarities = linear_kernel(Q_Train, con_a).flatten() 
#     #Sort top 10 index with similarity score
#     Top_index_doc = linear_similarities.argsort()[:-11:-1]
#     # sort by similarity score
#     linear_similarities.sort()
#     a = pd.DataFrame()
#     for i,index in enumerate(Top_index_doc):
#         a.loc[i,'index'] = str(index)
#         a.loc[i,'title'] = df['content'][index] ## Read File name with index from File_data DF
#     for j,simScore in enumerate(linear_similarities[:-11:-1]):
#         a.loc[j,'Score'] = simScore
#     return a


In [ ]:
# Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
df_final['content']=[entry.lower() for entry in df_final['content']]

In [ ]:
## data Cleaning for content data of news
df_final.content =df_final.content.replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True) #remove punctuation except
df_final.content =df_final.content.replace(to_replace='-',value=' ',regex=True)
df_final.content =df_final.content.replace(to_replace='\s+',value=' ',regex=True)    #remove new line
df_final.content =df_final.content.replace(to_replace='  ',value='',regex=True)                #remove double white space
df_final.content =df_final.content.apply(lambda x:x.strip())  # Ltrim and Rtrim of whitespace

In [ ]:
## data cleaning for title data 
df_final.title =df_final.title.replace(to_replace='[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
df_final.title =df_final.title.replace(to_replace='\s+',value=' ',regex=True)    #remove new line
df_final.title =df_final.title.replace(to_replace='  ',value='',regex=True)    #remove double white space
# df_news.title =df_news.title.apply(lambda x:x.strip())

In [ ]:
df_final

,title,content
0,Fecal Pollution Taints Water at Melbourne’s Be...,sydney australia the annual beach pilgrimage d...
1,Smithsonian Hears Call for Women’s Museum The ...,leading women from politics the arts and other...
2,NBC Will Bring Back ‘Will amp Grace’ The New Y...,in an era of revivals and reboots this may hav...
3,Bernie Ecclestone Is Replaced as Formula One C...,london bernie ecclestones longstanding leaders...
4,Pence Will Speak at Anti Abortion Rally The Ne...,washington vice president mike pence will spea...
...,...,...
7390,I interviewed the Ohio State attacker on the f...,abdul razak ali artan was sitting alone at a r...
7391,The Nationals have two years to win a World Se...,with baseballs winter meetings right here alon...
7392,Traces of explosives found on human remains fr...,cairo traces of explosives have been found on ...
7393,Australian police say they prevented bombings ...,melbourne australia police arrested seven susp...


In [ ]:
# df_news_cut = df_news_nodup[:12000]

In [ ]:
df_final.shape

(7395, 2)

In [ ]:
# df_final['Word tokenize']= [word_tokenize(entry) for entry in df_final.content]


In [ ]:
# df_news_cut

In [ ]:
# df_final = df_news_cut[['title', 'content']]

In [ ]:
# df_final 

In [ ]:
!pip install --upgrade tensorflow-gpu
#Install TF-Hub.
!pip install tensorflow-hub
!pip install seaborn

     |████████████████████████████████| 497.5 MB 16 kB/s 
     |████████████████████████████████| 462 kB 54.2 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import re, string
import os 
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.8.0
Eager mode:  True
Hub version:  0.12.0
GPU is available


In [ ]:
# ! curl -L -o 4.tar.gz "https://tfhub.dev/google/universal-sentence-encoder/4?tf-hub-format=compressed"


In [ ]:
#!mkdir /home/zettadevs/GoogleUSE Model
#! curl -L -o 4.tar.gz "https://tfhub.dev/google/universal-sentence-encoder/4?tf-hub-format=compressed" 
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
# module_path ="/home/zettadevs/GoogleUSEModel/USE_4"
%time model = hub.load(module_url)
#print ("module %s loaded" % module_url)

#Create function for using modeltraining
def embed(input):
    return model(input)

CPU times: user 13.3 s, sys: 2.95 s, total: 16.2 s
Wall time: 21.3 s


In [ ]:
# def plot_similarity(labels, features, rotation):
#     corr = np.inner(features, features)
#     sns.set(font_scale=1.2)
#     g = sns.heatmap(
#       corr,
#       xticklabels=labels,
#       yticklabels=labels,
#       vmin=0,
#       vmax=1,
#       cmap="YlOrRd")
#     g.set_xticklabels(labels, rotation=rotation)
#     g.set_title("Semantic Similarity")

In [ ]:
# def run_and_plot_U(messages_):
#     message_embeddings_ = embed(messages_)
#     plot_similarity(messages_, message_embeddings_, 90)

In [ ]:
# def SimilarityScore(messages):
#     message_embedding = embed(messages)
#     corr = np.inner(message_embedding,message_embedding)
#     #for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
#     #    print("Message: {}".format(messages[i]))
#     print(corr)

In [ ]:
df_final = df_final[:7000]

In [ ]:
df_final.shape

(7000, 2)

In [ ]:
ls =[]
chunksize =500
le =len(df_final.content)
for i in range(0,le,chunksize):
    if(i+chunksize > le): 
        chunksize= le;
        ls.append(chunksize)
    else:
        a =i+chunksize
        ls.append(a)
ls
j=0
for i in ls:
    directory = "/content/drive/MyDrive/IR/MODEL/" + str(i)
    if not os.path.exists(directory):
        os.makedirs(directory)
    directory = "/content/drive/MyDrive/IR/MODEL/" + str(i)
    print(j,i) 
    m=embed(df_final.content[j:i])
    exported_m = tf.train.Checkpoint(v=tf.Variable(m))
    exported_m.f = tf.function(
    lambda  x: exported_m.v * x,
    input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])

    tf.saved_model.save(exported_m,directory)
    j = i
    print(i)

0 500
INFO:tensorflow:Assets written to: /content/drive/MyDrive/IR/MODEL/500/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/IR/MODEL/500/assets


500
500 1000
INFO:tensorflow:Assets written to: /content/drive/MyDrive/IR/MODEL/1000/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/IR/MODEL/1000/assets


1000
1000 1500
INFO:tensorflow:Assets written to: /content/drive/MyDrive/IR/MODEL/1500/assets


...

6000
6000 6500
INFO:tensorflow:Assets written to: /content/drive/MyDrive/IR/MODEL/6500/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/IR/MODEL/6500/assets


6500
6500 7000
INFO:tensorflow:Assets written to: /content/drive/MyDrive/IR/MODEL/7000/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/IR/MODEL/7000/assets


7000


In [ ]:
ar =[]
for i in ls:
    directory = "/content/drive/MyDrive/IR/MODEL/" + str(i)
    if os.path.exists(directory):
        print(directory)
        imported_m = tf.saved_model.load(directory)
        a= imported_m.v.numpy()
        #print(a)
        ar.append(f'load{i}')
        exec(f'load{i} = a')  

/content/drive/MyDrive/IR/MODEL/500
/content/drive/MyDrive/IR/MODEL/1000
/content/drive/MyDrive/IR/MODEL/1500
/content/drive/MyDrive/IR/MODEL/2000
/content/drive/MyDrive/IR/MODEL/2500
/content/drive/MyDrive/IR/MODEL/3000
/content/drive/MyDrive/IR/MODEL/3500
/content/drive/MyDrive/IR/MODEL/4000
/content/drive/MyDrive/IR/MODEL/4500
/content/drive/MyDrive/IR/MODEL/5000
/content/drive/MyDrive/IR/MODEL/5500
/content/drive/MyDrive/IR/MODEL/6000
/content/drive/MyDrive/IR/MODEL/6500
/content/drive/MyDrive/IR/MODEL/7000


In [ ]:
lst = ['load500',
 'load1000',
 'load1500',
 'load2000',
 'load2500',
 'load3000',
 'load3500',
 'load4000',
 'load4500',
 'load5000',
 'load5500',
 'load6000',
 'load6500',
 'load7000']
#  'load7500',
#   'load8000'
#  'load8500',
#  'load9000',
#  'load9500',
#  'load10000',
#  'load10500',
#  'load11000',
#  'load11500',
#  'load12000',
#  'load12500',
#  'load13000',
#  'load13500',
#  'load14000',
#  'load14500',
#  'load15000',
#  'load15500',
#  'load16000',
#  'load16500',
#  'load17000',
#  'load17500',
#  'load18000',
#  'load18500',
#  'load19000',
#  'load19500',
#  'load20000',
#  'load20500',
#  'load21000',
#  'load21500',
#  'load22000',
#  'load22500',
#  'load23000',
#  'load23500',
#  'load24000',
#  'load24500',
#  'load25000',
#  'load25500',
#  'load26000',
#  'load26500',
#  'load27000',
#  'load27500',
#  'load28000',
#  'load28500',
#  'load29000',
#  'load29500',
#  'load29661']

In [ ]:
for i in lst:
  txt = i.replace("'", "")
  print(txt)

# txt

load500
load1000
load1500
load2000
load2500
load3000
load3500
load4000
load4500
load5000
load5500
load6000
load6500
load7000


In [ ]:
con_a =np.concatenate((load500, load1000, load1500,
load2000,
load2500,
load3000,
load3500,
load4000,
load4500,
load5000,
load5500,
load6000,
load6500,
load7000))



# load8500,
# load9000,
# load9500,
# load10000,
# load10500,
# load11000,
# load11500,
# load12000,
# load12500,
# load13000,
# load13500,
# load14000,
# load14500,
# load15000,
# load15500,
# load16000,
# load16500,
# load17000,
# load17500,
# load18000,
# load18500,
# load19000,
# load19500,
# load20000,
# load20500,
# load21000,
# load21500,
# load22000,
# load22500,
# load23000,
# load23500,
# load24000,
# load24500,
# load25000,
# load25500,
# load26000,
# load26500,
# load27000,
# load27500,
# load28000,
# load28500,
# load29000,
# load29500,
# load29661
con_a.shape

(7000, 512)

In [ ]:
## training the model
Model_USE= embed(df_final.content[:7000])

In [ ]:
exported = tf.train.Checkpoint(v=tf.Variable(Model_USE))
exported.f = tf.function(
    lambda  x: exported.v * x,
    input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])

tf.saved_model.save(exported,'/content/drive/MyDrive/IR/MODEL')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/IR/MODEL/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/IR/MODEL/assets


In [ ]:
## Load Model
imported = tf.saved_model.load('/content/drive/MyDrive/IR/MODEL/')
loadedmodel =imported.v.numpy()
loadedmodel.shape

(7000, 512)

In [ ]:
def SearchDocument(query):
    q =[query]
    # embed the query for calcluating the similarity
    Q_Train =embed(q)
    # Calculate the Similarity
    linear_similarities = linear_kernel(Q_Train, con_a).flatten() 
    #Sort top 10 index with similarity score
    Top_index_doc = linear_similarities.argsort()[:-11:-1]
    # sort by similarity score
    linear_similarities.sort()
    a = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        a.loc[i,'index'] = str(index)
        a.loc[i,'File_Name'] = df_final['title'][index] ## Read File name with index from File_data DF
    for j,simScore in enumerate(linear_similarities[:-11:-1]):
        a.loc[j,'Score'] = simScore
    return a

In [ ]:
SearchDocument('Trump')

,index,File_Name,Score
0,3103,Donald Trump entertains idea he might not actu...,0.491851
1,1210,Bill Kristol Whatever One Thinks About Donald ...,0.484306
2,885,How Anti Trump Violence Could Elect Him The Da...,0.465091
3,3328,When Donald Trump Lavished Praise on Hillary C...,0.464272
4,4480,Trump Declines To Name Women He’d Put In His ...,0.459564
5,1031,Cruz Voters Want a Real Conservative Not Trump...,0.457446
6,1324,Donald Trump ’I Will Always Tell You the Truth’,0.447334
7,2033,Scarborough Press Underestimating Trump Is Hel...,0.443826
8,896,Peter King on Trump’s Carrier Deal ’There Is a...,0.443414
9,3193,Donald Trump faces a debate trap,0.441725


In [ ]:
i = 1000

f'load{i}'

'load1000'